<a href="https://colab.research.google.com/github/RedPanda54/Machine_Learning_study/blob/main/Linear_models_for_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**데이터 전처리**

In [96]:
import numpy as np
import pandas as pd

Url = 'https://raw.githubusercontent.com/RedPanda54/Machine_Learning_study/main/auto-mpg.csv'
data = pd.read_csv(Url)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


In [97]:
# 'car name' feature 무시
data.drop('car name', axis=1, inplace=True)

In [98]:
# horsepower type 변환
data['horsepower'] = pd.to_numeric(data['horsepower'], errors='coerce')
data.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower      float64
weight            int64
acceleration    float64
model year        int64
origin            int64
dtype: object

In [99]:
# 결측치 확인
data.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
dtype: int64

In [100]:
mean_horsepower = data['horsepower'].mean()
data['horsepower'].fillna(mean_horsepower, inplace=True)

In [101]:
data.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
dtype: int64

**문제 1**

In [102]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [103]:
# 독립 변수(X)와 종속 변수(y) 분리
X = data.drop('mpg', axis=1)
y = data['mpg']

In [104]:
# 다항 회귀와 k-fold cross-validation을 통한 성능 비교
best_score = float('inf')
best_degree = 0

for degree in range(1, 6):
    # 다항 특성 생성
    poly_features = PolynomialFeatures(degree=degree)
    X_poly = poly_features.fit_transform(X)

    # 선형 회귀 모델 학습
    model = LinearRegression()
    model.fit(X_poly, data['mpg'])

    # k-fold cross-validation을 통한 평가
    scores = cross_val_score(model, X_poly, y, scoring='neg_mean_squared_error', cv=5)
    avg_mse = -scores.mean()

    # 성능 비교
    if avg_mse < best_score:
        best_score = avg_mse
        best_degree = degree

In [73]:
# 가장 성능이 좋은 다항식 차수를 출력
print(f"Best degree: {best_degree}")
print(f"Best MSE: {best_score}")

Best degree: 2
Best MSE: 14.104416766014703


**문제 2**

In [86]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

In [87]:
# 데이터 전처리
# 이전에 수행한 데이터 전처리 코드를 활용
X = data['horsepower'].values.reshape(-1, 1)
y = data['mpg'].values

In [88]:
# 3차 다항식 특성 생성
poly_features = PolynomialFeatures(degree=3)
X_poly = poly_features.fit_transform(X)

In [89]:
# data split
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

In [94]:
# Grid Search를 위한 Ridge 모델과 파라미터 그리드 설정
ridge = Ridge()
param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 50, 100, 200]}

# Grid Search를 수행하여 최적의 Ridge 파라미터 값 찾기
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 Ridge 파라미터와 그에 해당하는 모델 생성
best_alpha = grid_search.best_params_['alpha']
best_model = Ridge(alpha=best_alpha)
best_model.fit(X_train, y_train)

# 테스트 세트에서의 예측값 계산
y_pred = best_model.predict(X_test)

# 테스트 세트에서의 MSE 계산
mse = mean_squared_error(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.92819e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.0201e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.03926e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.05808e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning

In [95]:
print(f"Best alpha: {best_alpha}")
print(f"Best MSE: {mse}")

Best alpha: 50
Best MSE: 13.787548745800976
